In [2]:
from telebot import TeleBot
import textdistance
from parsing_funcs import *
import prettytable as pt
import pickle
import io
from telebot.types import InlineKeyboardMarkup, ReplyKeyboardMarkup, KeyboardButton
import os

In [3]:
os.environ['DATA_PROCESSES_BOT_TOKEN'] = '5078166387:AAEEm_8z6cvqqon2MzTtcX2AlGmcLpKtph0'

In [4]:
CHAT_ID = '1323215' # The chat ID where this message should be sent to - my chat with bot
TOKEN = os.environ['DATA_PROCESSES_BOT_TOKEN'] # The one you got from @BotFather while creating the bot
bot = TeleBot(TOKEN)

In [5]:
housing_faq = parse_housing_faq()
housing_faq.head()

,question,answer
0,How can I apply for accommodation?,You will need to fill out a form in your onlin...
1,Will I sign a formal rental agreement?,"Yes, you will. Each student signs a rental agr..."
2,Can I contact the Student Housing Office manag...,Yes. If you have any questions about your acco...
3,How will I pay for accommodation? Via the univ...,If the contract (rental or accommodation contr...
4,Will there be a security deposit?,All accommodation options require a security d...


In [6]:
programmes = parse_programmes()
programmes.head()

,programme,faculty,link,type,department,location,duration,schedule,language
0,07.00.00 Architecture,Urban Planning,https://www.hse.ru/en/ba/up/,bachelors,Faculty of Urban and Regional Development,Moscow,5 years,Full-time,Russian and English*
1,06.00.00 Biology,Cell and Molecular Biotechnology,https://www.hse.ru/en/ba/cmb/,bachelors,Faculty of Biology and Biotechnology,Moscow,4 years,Full-time,Russian and English*
2,06.00.00 Biology,Cognitive Neurobiology,https://www.hse.ru/en/ba/cmb/,bachelors,Faculty of Biology and Biotechnology,Moscow,4 years,Full-time,Russian and English*
3,58.00.00 Asian and African Studies,Asian and African Studies,https://www.hse.ru/en/ba/oriental/,bachelors,Faculty of World Economy and International Aff...,Moscow,5 years,Full-time,Russian and English*
4,58.00.00 Asian and African Studies,Asian and African Studies,https://www.hse.ru/en/ba/oriental/,bachelors,Saint-Petersburg School of Social Sciences and...,Saint Petersburg,5 years,Full-time,Russian and English*


In [7]:
rankings = parse_rankings()
rankings = rankings.fillna('')
rankings.head()

,ranking,subject,place_world,place_russia
0,"QS World University Rankings, 2021/2022",,305,7
1,"QS – World University Rankings by subject, 2021",Politics & International Studies,45,2
2,"QS – World University Rankings by subject, 2021",Sociology,50,1
3,"QS – World University Rankings by subject, 2021",Education,64,1
4,"QS – World University Rankings by subject, 2021",Economics & Econometrics,76,1


In [8]:
living = parse_costs(which='living')
living.head()

,Programme,Tuition (per year)
0,Housing (HSE dormitory for full scholarship ca...,around 1100 – 1400 RUB
1,Housing (rent for fee-paying candidates)\nFYI:...,around 25 000 – 30 000 RUB
2,Food,around 16 000 RUB
3,Transportation,around 1600 RUB
4,Phone,around 600 RUB


In [9]:
for a, b in living.values:
    print(a, b)

Housing (HSE dormitory for full scholarship candidates) around 1100 – 1400 RUB
Housing (rent for fee-paying candidates)
FYI: Cost of living index in Moscow is 53.44% lower than in Amsterdam.
Rent in Moscow is, on average, 55.93% lower than in Amsterdam. *as of December 2020 around 25 000 – 30 000 RUB
Food around 16 000 RUB
Transportation around 1600 RUB
Phone around 600 RUB


In [10]:
rankings[rankings['subject'] == '']

,ranking,subject,place_world,place_russia
0,"QS World University Rankings, 2021/2022",,305,7
25,"QS – Top 50 Under 50, 2020",,31,1
34,"Academic Ranking of World Universities, 2021",,601–700,4–5
35,"THE World University Rankings, 2021/2022",,301–350,3–4
36,"THE Young University Rankings, 2021",,57,1
37,"THE World Reputation Rankings, 2021",,126-150,2-4
48,U.S.News & World Report Best Global Universiti...,,568,7
55,"Three University Missions Ranking, 2021",,120,4


In [15]:
@bot.message_handler(content_types=['text'])
def get_user_command_choice(message, housing_faq_questions_similarity_barrier=0, programmes_similarity_barrier=0):
    if message.text == "/accomodation":
        bot.send_message(message.from_user.id, "Please send me your question ↓")
        
        # analyzing the question
        bot.register_next_step_handler(
            message, 
            lambda x: get_accomodation_answer(
                x, 
                housing_faq_questions_similarity_barrier=housing_faq_questions_similarity_barrier
            )
        )
    elif message.text == "/costs":
        get_costs_answer(message)
    elif message.text == "/ranking":
        get_ranking_answer(message)
        # bot.send_message(message.from_user.id, "Not supported yet")
    elif message.text == "/programmes":
        bot.send_message(message.from_user.id, "Which faculty are you interested in?")
        
        # analyzing the question
        bot.register_next_step_handler(
            message,
            lambda x: get_programmes_answer(
                x, 
                programmes_similarity_barrier=programmes_similarity_barrier
            )
        )
    else:
        bot.send_message(message.from_user.id, "Please choose one of the commands in the menu for me to help you!")

In [16]:
def get_sentences_similarity(q1, q2):
    return textdistance.hamming.normalized_similarity(q1, q2)

In [17]:
def get_accomodation_answer(message, housing_faq_questions_similarity_barrier=0): 
    
    # go to the necessary table within the database & fetch most recent date info: housing_faq
    # hosing_faq = <QUERY>
    
    housing_faq['score'] = housing_faq['question'].apply(lambda x: get_sentences_similarity(message.text, x))
    max_score = housing_faq['score'].max()
    
    if max_score < housing_faq_questions_similarity_barrier:
        reply = "Sorry, we do not seem to know the answer. Please contact test@gmail.com"
    else:
        # returning the first answer in case there are multiple
        reply = housing_faq[housing_faq['score'] == max_score]['answer'].values[0]
    
    bot.send_message(message.from_user.id, reply)

In [18]:
def send_dataframe(df, chat_id, file_name='file', format_='csv'):
#     my_bytes = pickle.dumps(df, protocol=4)
#     file_obj = io.BytesIO(my_bytes)
#     file_obj.name = file_name
    file_name = f"{file_name}.{format_}"
    df.to_csv(file_name)
    bot.send_document(chat_id, open(file_name, 'r'))

In [19]:
def get_programmes_answer(message, programmes_similarity_barrier=0):
    
    def get_language_answer(message, faculty):
        """given faculty & language info, return the list of programmes"""
        language_input = message.text.lower().strip().replace(' ', '')
        
        # checking each possible part
        for part, full in zip(['rus', 'eng', 'ger', 'pol', 'STOP'], ['Russian', 'English', 'German', 'Polish', 'STOP']):
            if part in language_input:
                actual_language = full
                relevant_programmes = programmes[
                    (programmes['faculty'] == faculty)
                    & (programmes['language'].str.contains(full))
                ]
                break # found the language!
            if part == 'STOP':
                bot.send_message(message.from_user.id, "Did not find such language. Please try again.")
                return None # finish the function
        
        # should send the filtered table now
        send_dataframe(df=relevant_programmes, chat_id=message.from_user.id, file_name='programmes')
#         bot.send_message(message.from_user.id, "REACHED THE END!")
                
    
    # go to the necessary table within the database & fetch most recent date info: programmes
    # programmes = <QUERY>
    
    programmes['score'] = programmes['faculty'].apply(lambda x: get_sentences_similarity(message.text, x))
    max_score = programmes['score'].max()
    print(max_score)
    
    if max_score < programmes_similarity_barrier:
        reply = "Sorry, couldn't find this faculty. Take a look at 'Urban Planning' maybe :)"
        bot.send_message(message.from_user.id, reply)
    else:
        # the desired faculty
        faculty = programmes[programmes['score'] == max_score]['faculty'].values[0]
        # asking for language
        bot.send_message(message.from_user.id, "Which language do you want to study in?")
        
        # analyzing the question
        bot.register_next_step_handler(
            message,
            lambda x: get_language_answer(
                x, 
                faculty=faculty
            )
        )

In [20]:
def get_ranking_answer(message):
    
    # go to the necessary table within the database & fetch most recent date info: ranking
    # ranking = <QUERY>
    
    # gathering the 3-button menu
    def offer_choice_markup():
        user_markup = ReplyKeyboardMarkup(True, True)
        for text in ['university ranking', 'ranking by faculty', 'ranking by subject']:
            button = KeyboardButton(text=text)       
            user_markup.add(button)
        return user_markup

    bot.send_message(
        message.from_user.id,
        text=f"""Please choose one of the menu options ↓""", 
        parse_mode='markdown', 
        reply_markup=offer_choice_markup()
    )  

In [21]:
def get_costs_answer(message):
    
    # go to the necessary table within the database & fetch most recent date info: living
    # living = <QUERY>
    
    def get_fees_answer(message):
        if 'living' in message.text:
            reply = '\n'.join([f"{name}: {value}" for name, value in living.values])
            bot.send_message(message.from_user.id, reply)
            # bot.send_message(message.from_user.id, "Not supported yet")
        elif 'tuition' in message.text:
            bot.send_message(
                message.from_user.id, 
                "Please review this link: https://admissions.hse.ru/en/graduate-apply/fees"
            )
    
    # gathering the 2-button menu
    def offer_choice_markup():
        user_markup = ReplyKeyboardMarkup(True, True)
        for text in ['🏡 living fees', '📚 tuition fees']:
            button = KeyboardButton(text=text)       
            user_markup.add(button)
        return user_markup

    # sending the menu
    bot.send_message(
        message.from_user.id,
        text=f"""Please choose one of the menu options ↓""", 
        parse_mode='markdown', 
        reply_markup=offer_choice_markup()
    )  
    
    # analyzing the question
    bot.register_next_step_handler(
        message,
        get_fees_answer
    )

In [22]:
bot.polling(none_stop=True, interval=0)